## This notebook retrieves data from three relevant tables and combines the features we care about into one dataframe. We do this, because merging all of the data is very costly.


**Description of columns**


- **user_id**: Unique identifier for the user (from triplets table).  
- **song_id**: Echo Nest song ID. Multiple tracks can share the same song_id if they are different versions of the same song. However, we removed these duplicates from the track_metadata table.  
- **plays**: Number of times the user played the song (from triplets table).  
- **track_id**: The Echo Nest ID of this particular track on which the analysis was done. Unique identifier for each track within the Million Song Dataset.  
- **title**: Title of the song.  
- **release**: Release (album name) from which the track was taken.  
- **artist_id**: Unique identifier for the artist. Matches the artist_id field in the track_metadata table and artist_term table.  
- **artist_mbid**: MusicBrainz ID (MBID) for the artist.  
- **artist_name**: Name of the artist.  
- **duration**: Duration of the track in seconds.  
- **artist_familiarity**: EchoNest numerical estimation of how familiar a song’s artist currently is to the world.  
- **artist_hotttnesss**: EchoNest numerical estimation of how hottt a song’s artist currently is.  
- **year**: The year the track was released according to MusicBrainz.  
- **track_7digitalid**: Identifier linking the song to the 7digital online music catalog.  
- **shs_perf**: Performance number on the SecondHandSongs website. Default value is -1.  
- **shs_work**: Clique numbers from the SecondHandSongs train and test files. If the work is known by MillionSongs, then the number is positive.  
- **majority_genre**: The most common or dominant genre associated with the artist or track (derived field).  
- **minority_genre**: A secondary or less frequent genre associated with the artist or track (derived field).  

### Retrieve Data

In [ ]:
import sqlite3

import pandas as pd

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
user_song_play_path = "/content/drive/MyDrive/DSE/DSE 220 Final Project/Data/train_triplets.txt"
track_metadata_path = "/content/drive/MyDrive/DSE/DSE 220 Final Project/Data/track_metadata.db"
genre_labels_path = "/content/drive/MyDrive/DSE/DSE 220 Final Project/Data/msd_tagtraum_cd2.cls"

In [ ]:
# Songs can have multiple track ids, so we retrieve only one
# Once we have only one song we are no longer concerned about "tracks"
track_metadata_conn = sqlite3.connect(track_metadata_path)
track_metadata_sql_query = "SELECT * FROM songs"
song_metadata_df = pd.read_sql_query(
    track_metadata_sql_query,
    track_metadata_conn
).drop_duplicates(subset=['song_id'])
track_metadata_conn.close()

user_song_play_df = pd.read_csv(
    user_song_play_path,
    sep='\t',
    names=['user_id', 'song_id', 'plays'],
)

genre_labels_df = pd.read_csv(
    genre_labels_path,
    sep='\t',
    names=['track_id', 'majority_genre', 'minority_genre'],
    skiprows=7
)

song_metadata_with_genre_df = pd.merge(
    song_metadata_df,
    genre_labels_df,
    on='track_id',
    how='left'  # so we can keep all rows
)

### Combine Data

In [ ]:
songs_df_full = pd.merge(user_song_play_df, song_metadata_with_genre_df, on='song_id')
# Calculate the amount of time a user spent listening to a particular song
songs_df_full['tot_track_time'] = songs_df_full['duration'] * songs_df_full['plays']
songs_df = songs_df_full[['user_id', 'song_id', 'plays', 'track_id', 'title', 'release',
                          'artist_id', 'duration', 'artist_name',
                          'artist_familiarity', 'artist_hotttnesss', 'year', 'majority_genre', 'minority_genre']]

In [ ]:
print(songs_df_full.columns)
print(songs_df.columns)

Index(['user_id', 'song_id', 'plays', 'track_id', 'title', 'release',
       'artist_id', 'artist_mbid', 'artist_name', 'duration',
       'artist_familiarity', 'artist_hotttnesss', 'year', 'track_7digitalid',
       'shs_perf', 'shs_work', 'majority_genre', 'minority_genre',
       'tot_track_time'],
      dtype='object')
Index(['user_id', 'song_id', 'plays', 'track_id', 'title', 'release',
       'artist_id', 'duration', 'artist_name', 'artist_familiarity',
       'artist_hotttnesss', 'year', 'majority_genre', 'minority_genre'],
      dtype='object')


In [ ]:
user_total_plays = songs_df.groupby('user_id', as_index=False)['plays'].sum().rename(
    columns={'plays': 'total_play_count'})

user_unique_songs = songs_df.groupby('user_id', as_index=False)['song_id'].count().rename(
    columns={'song_id': 'unqiue_song_count'})
user_unique_artists = songs_df.groupby('user_id', as_index=False)['artist_id'].count().rename(
    columns={'artist_id': 'unqiue_artist_count'})
user_total_time = songs_df.groupby('user_id', as_index=False)['tot_track_time'].sum().rename(
    columns={'tot_track_time': 'total_play_time'})

user_fav_artist = songs_df.groupby('user_id', as_index=False)['artist_id'].value_counts()
user_fav_artist = user_fav_artist.loc[user_fav_artist.groupby('user_id')['count'].idxmax()].rename(
    columns={'artist_id': 'fav_artist', "count": "fav_artist_plays"})

user_fav_maj_genre = songs_df.groupby('user_id', as_index=False)['majority_genre'].value_counts()
user_fav_maj_genre = user_fav_maj_genre.loc[user_fav_maj_genre.groupby('user_id')['count'].idxmax()].rename(
    columns={'majority_genre': 'fav_maj_genre', "count": "fav_maj_count"})
user_fav_min_genre = songs_df.groupby('user_id', as_index=False)['minority_genre'].value_counts()
user_fav_min_genre = user_fav_min_genre.loc[user_fav_min_genre.groupby('user_id')['count'].idxmax()].rename(
    columns={'minority_genre': 'fav_min_genre', "count": "fav_min_count"})

user_fav_year = songs_df.groupby('user_id', as_index=False)['year'].value_counts()
user_fav_year = user_fav_year[user_fav_year['year'] != 0]
user_fav_year = user_fav_year.loc[user_fav_year.groupby('user_id')['count'].idxmax()].rename(
    columns={'year': 'fav_year', "count": "year_count"})

user_fav_song = songs_df.loc[songs_df.groupby('user_id')['plays'].idxmax()]
user_fav_song = user_fav_song[['user_id', 'song_id', 'plays', 'duration', 'tot_track_time']].rename(
    columns={'song_id': 'fav_song', "plays": "fav_song_plays", 'duration': 'fav_song_duration',
             'tot_track_time': 'tot_fav_song_play_time'})

#### Create final df pulling relevant features

In [ ]:
users_df = pd.merge(user_total_plays, user_total_time, on="user_id")
users_df = pd.merge(users_df, user_unique_artists, on="user_id")
users_df = pd.merge(users_df, user_unique_songs, on="user_id")
users_df = pd.merge(users_df, user_fav_maj_genre, on="user_id")
users_df = pd.merge(users_df, user_fav_min_genre, on="user_id")
users_df = pd.merge(users_df, user_fav_year, on="user_id")
users_df = pd.merge(users_df, user_fav_artist, on="user_id")
users_df = pd.merge(users_df, user_fav_song, on="user_id")

### Save processed data frames for other notebooks to process further and use

In [ ]:
users_df.to_csv('/content/drive/MyDrive/DSE/DSE 220 Final Project/Data/users_df.csv', index=False)

In [ ]:
songs_df_full.to_csv('/content/drive/MyDrive/DSE/DSE 220 Final Project/Data/songs_df_full.csv', index=False)